In [1]:
import io
import os
import random
from pathlib import Path

In [2]:
%pwd

'c:\\Users\\Asus\\Machine_learning\\LLM\\Projects\\BIRD_benchmark_SQL\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Asus\\Machine_learning\\LLM\\Projects\\BIRD_benchmark_SQL'

In [5]:
import os
from Bird_bench_SQL import logger

In [6]:
from pathlib import Path
from typing import Optional
from dataclasses import dataclass

In [7]:
@dataclass(frozen=True)
class DatabaseAndModelConfig:
    SQLite_database_path    : str
    Model_name              : str 
    temperature             : int 
    api_key                 : Optional[str]

In [8]:
from dotenv import load_dotenv
from Bird_bench_SQL.constants import *
from Bird_bench_SQL.utils.common import read_yaml,create_directories
from Bird_bench_SQL.config.configuration import ConfigurationManager

In [16]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath    = CONFIG_FILE_PATH,
                 params_filepath    = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_database_and_model_config(self) -> DatabaseAndModelConfig:
        config =  self.config.database_and_model
        params = self.params 
        
        logger.info('Database and model config initialized')
        load_dotenv()
        data_procesing_config = DatabaseAndModelConfig(SQLite_database_path = config.database_path,
                                                       Model_name           = params.MODEL_NAME,
                                                       temperature          = params.TEMPERATURE,
                                                       api_key              = os.getenv("GROQ_API_KEY")) 
        logger.info("database and model config finished") 
        return data_procesing_config        


In [20]:

manager     = ConfigurationManager()
config      = manager.get_database_and_model_config()

[2024-09-09 17:58:48,079: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-09 17:58:48,083: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-09 17:58:48,083: INFO: common: created directory at: artifacts]
[2024-09-09 17:58:48,083: INFO: 715277999: Database and model config initialized]
[2024-09-09 17:58:48,090: INFO: 715277999: database and model config finished]


In [32]:
database_path = Path(config.SQLite_database_path)
query = "SELECT T1.title FROM movie AS T1 INNER JOIN movie_keywords AS T2 ON T1.movie_id = T2.movie_id INNER JOIN keyword AS T3 ON T2.keyword_id = T3.keyword_id WHERE T3.keyword_name = 'extremis'"

In [19]:
import os
import sqlite3
from langchain_groq import ChatGroq
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase

In [77]:
class DatabaseAndModel:

    def __init__(self,config = DatabaseAndModelConfig):
        self.config = config

    def database_and_model_setup(self):
        logger.info("Model setup initialized")
        llm     = ChatGroq(model        = self.config.Model_name,
                           temperature  = self.config.temperature,
                           api_key      = self.config.api_key,)
        
        logger.info(f"model----{(self.config.Model_name).split('/')[-1]}----created")
        conn    = sqlite3.connect(self.config.SQLite_database_path) 
        logger.info(f"connection----{(self.config.SQLite_database_path).split('/')[-1]}----created")

        engine  = create_engine(f"sqlite:///{self.config.SQLite_database_path}")
        logger.info(f"engine----{(self.config.SQLite_database_path).split('/')[-1]}----created")

        db      = SQLDatabase(engine) 
        logger.info(f"database----created")
        return llm,conn,db 


In [78]:
try:
    
    manager             = ConfigurationManager()
    config              = manager.get_database_and_model_config()
    datamodel_and_model = DatabaseAndModel(config=config)
    llm,conn,engine  = datamodel_and_model.database_and_model_setup()

except Exception as e:
    raise e

[2024-09-09 18:49:57,638: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-09 18:49:57,640: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-09 18:49:57,642: INFO: common: created directory at: artifacts]
[2024-09-09 18:49:57,643: INFO: 715277999: Database and model config initialized]
[2024-09-09 18:49:57,645: INFO: 715277999: database and model config finished]
[2024-09-09 18:49:57,645: INFO: 3302737611: Model setup initialized]
[2024-09-09 18:49:58,154: INFO: 3302737611: model----llama3-70b-8192----created]
[2024-09-09 18:49:58,155: INFO: 3302737611: connection----movies_4.sqlite----created]
[2024-09-09 18:49:58,156: INFO: 3302737611: engine----movies_4.sqlite----created]
[2024-09-09 18:49:58,180: INFO: 3302737611: database----created]
